In [1]:
import torch as tr
import numpy as np

In [2]:
input = tr.from_numpy(np.array([[ 0.52, -1.37,  0.11],
        [ 1.24,  0.89, -0.47],
        [-0.91,  0.32,  0.75],
        [ 0.04, -0.58, -0.12],
        [ 0.67,  1.93, -0.34],
        [-1.12, -0.45,  1.08],
        [ 0.88,  0.05,  0.56],
        [-0.63,  0.49, -0.87],
        [ 1.01, -0.16,  0.92],
        [ 0.39,  0.72, -0.65]], dtype="float32"))
target = tr.from_numpy(np.array([[ 0.15, -1.02],
        [ 0.84,  0.31],
        [-0.55,  0.69],
        [ 1.21, -0.48],
        [-0.17,  0.43],
        [ 0.64, -1.13],
        [ 0.29,  0.77],
        [ 0.92, -0.35],
        [-0.68,  1.11],
        [ 0.56,  0.08]] , dtype="float32"))

## Dataset and DataLoader
TensorDataset , this helps to allow accesss to rows in tensors as tuples

DataLoader , this helps to split data into batches of predefined sizeswhile training.

In [3]:
from torch.utils.data import TensorDataset

train_ds = TensorDataset(input, target)
train_ds[0:3]


(tensor([[ 0.5200, -1.3700,  0.1100],
         [ 1.2400,  0.8900, -0.4700],
         [-0.9100,  0.3200,  0.7500]]),
 tensor([[ 0.1500, -1.0200],
         [ 0.8400,  0.3100],
         [-0.5500,  0.6900]]))

In [4]:
from torch.utils.data import DataLoader

batch_size = 2
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [5]:
for xb, yb in train_dl:
  print("batche:")
  print(xb)
  print(yb)


batche:
tensor([[-0.6300,  0.4900, -0.8700],
        [ 0.3900,  0.7200, -0.6500]])
tensor([[ 0.9200, -0.3500],
        [ 0.5600,  0.0800]])
batche:
tensor([[-0.9100,  0.3200,  0.7500],
        [-1.1200, -0.4500,  1.0800]])
tensor([[-0.5500,  0.6900],
        [ 0.6400, -1.1300]])
batche:
tensor([[ 1.2400,  0.8900, -0.4700],
        [ 0.0400, -0.5800, -0.1200]])
tensor([[ 0.8400,  0.3100],
        [ 1.2100, -0.4800]])
batche:
tensor([[ 1.0100, -0.1600,  0.9200],
        [ 0.8800,  0.0500,  0.5600]])
tensor([[-0.6800,  1.1100],
        [ 0.2900,  0.7700]])
batche:
tensor([[ 0.6700,  1.9300, -0.3400],
        [ 0.5200, -1.3700,  0.1100]])
tensor([[-0.1700,  0.4300],
        [ 0.1500, -1.0200]])


# nn.Linear
Instead of manually initialing weights , we can use nn.linear class from PyTorch to define the model

In [6]:
import torch.nn as nn
model = nn.Linear(3,2) #In this we are nmentioning there  are 3 inputs to the modela nd 2 outputs received from the model
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.0876, -0.1057, -0.0228],
        [-0.4306, -0.1434, -0.3994]], requires_grad=True)
Parameter containing:
tensor([ 0.0786, -0.3066], requires_grad=True)


# .parameters()
this function returns list of Weights and Biase MAtrices present in the model

For this project we willl have only One Weight and Bias Matrix

In [7]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0876, -0.1057, -0.0228],
         [-0.4306, -0.1434, -0.3994]], requires_grad=True),
 Parameter containing:
 tensor([ 0.0786, -0.3066], requires_grad=True)]

In [8]:
initial_predicton = model(input)
print(initial_predicton)

tensor([[ 0.2664, -0.3780],
        [ 0.1039, -0.7804],
        [-0.0520, -0.2602],
        [ 0.1461, -0.1927],
        [-0.0589, -0.7360],
        [ 0.0034, -0.1912],
        [ 0.1377, -0.9163],
        [-0.0086,  0.2419],
        [ 0.1631, -1.0859],
        [ 0.0515, -0.3181]], grad_fn=<AddmmBackward0>)


# nn.functional module

This module has many Loss functions and other useful utilities


In [9]:
import torch.nn.functional as F

mse = F.mse_loss

loss = mse(initial_predicton,target)
print(loss)

tensor(0.8605, grad_fn=<MseLossBackward0>)


Object `torch.optim.SGD()` not found.


# Optimizers:
instead of we updating the weight smanually, this takes care of the weight updation!
SGD: Stochastic Gradient Descent (samples are taken as batches instead of oen single group)

##NOTE:
model.parameters() is sent as arguement, so that the optimizer knows about the matrices which should be modified during the update steps.

learning rate is also inputted to control the rate in which model learns



In [10]:
optimizer = tr.optim.SGD(model.parameters() , lr=1e-5)

In [15]:
def fit(num,model,loss_fn,optimizer):
  for i in range(num):
     for xb,yb in train_dl:
      predictions = model(xb)
      loss = loss_fn(predictions,yb)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

  if (i+1) % 10 == 0:
      print('Epoch [{}/{}], Loss: {:.4f}'.format(i+1, num, loss.item()))




In [21]:
fit(20000,model,mse,optimizer)

Epoch [20000/20000], Loss: 0.3901


In [20]:
print(model(input))

tensor([[ 0.2973, -0.3359],
        [ 0.1732, -0.4237],
        [-0.0148, -0.2090],
        [ 0.2029, -0.1470],
        [ 0.0224, -0.3430],
        [ 0.0206, -0.2351],
        [ 0.1543, -0.6472],
        [ 0.1050,  0.2669],
        [ 0.1582, -0.8019],
        [ 0.1409, -0.1109]], grad_fn=<AddmmBackward0>)


In [18]:
print(target)

tensor([[ 0.1500, -1.0200],
        [ 0.8400,  0.3100],
        [-0.5500,  0.6900],
        [ 1.2100, -0.4800],
        [-0.1700,  0.4300],
        [ 0.6400, -1.1300],
        [ 0.2900,  0.7700],
        [ 0.9200, -0.3500],
        [-0.6800,  1.1100],
        [ 0.5600,  0.0800]])
